# 02 - Data Processing

In [1]:
import sys
sys.executable

'/usr/local/bin/python'

## Imports

In [2]:
import numpy as np
import pandas as pd

import stock_utils.constants as cnst
import stock_utils.data as sd

pd.set_option('display.max_columns', None)

NSE_DATA_DIR = PosixPath('../data/NSE') | Valid: True
SCREENER_EPS_DATA_DIR = PosixPath('../data/Screener.in/EPS') | Valid: True
PROCESSED_DATA_DIR = PosixPath('../data/processed') | Valid: True


## Constants

In [3]:
stock_symbols = sd.get_all_stock_symbols(
    cnst.NSE_DATA_DIR
)

stock_symbols

['DEEPAKFERT',
 'HDFCBANK',
 'HDFCSML250',
 'INDIGOPNTS',
 'ITBEES',
 'JUBLFOOD',
 'NH']

In [4]:
STOCK_SYMBOL = stock_symbols[2]
STOCK_SYMBOL

'HDFCSML250'

## Data loading

In [5]:
stock_df = pd.read_parquet(cnst.NSE_DATA_DIR.joinpath(STOCK_SYMBOL, "consolidated.parquet"))
stock_df.columns = [
    'Date', 'Series',
    'Open', 'High', 'Low', 'Prev Close', 'LTP', 'Close',
    'VWAP', '52W H', '52W L',
    'Volume', 'Value', '#Trades'
] + (["PE"] if "PE" in stock_df.columns else [])
stock_df

,Date,Series,Open,High,Low,Prev Close,LTP,Close,VWAP,52W H,52W L,Volume,Value,#Trades
0,2023-02-21,EQ,92.00,94.00,92.00,91.31,93.50,93.48,93.22,94.0,92.00,2565,239114.97,38
1,2023-02-22,EQ,93.48,93.50,90.74,93.48,91.75,92.02,92.25,94.0,90.74,12207,1126142.91,172
2,2023-02-23,EQ,92.02,92.02,89.75,92.02,90.50,90.59,90.76,94.0,89.75,6153,558445.34,102
3,2023-02-24,EQ,90.76,90.76,90.50,90.59,90.60,90.59,90.62,94.0,89.75,1944,176173.22,47
4,2023-02-27,EQ,92.50,95.80,87.50,90.59,89.85,89.97,89.74,95.8,87.50,6801,610333.75,227
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329,2024-06-24,EQ,172.49,172.49,169.05,171.28,171.21,170.92,170.91,176.0,103.40,300649,51383833.52,6275
330,2024-06-25,EQ,171.78,173.06,170.92,170.92,171.30,171.42,171.94,176.0,103.40,381223,65548651.83,4284
331,2024-06-26,EQ,171.43,172.37,171.00,171.42,171.35,171.45,171.54,176.0,103.40,230914,39610803.32,4389
332,2024-06-27,EQ,171.54,172.54,169.45,171.45,170.70,170.42,170.80,176.0,103.40,447571,76442930.84,7430


In [6]:
stock_df.info(memory_usage = "deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 334 entries, 0 to 333
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Date        334 non-null    datetime64[ns]
 1   Series      334 non-null    object        
 2   Open        334 non-null    float64       
 3   High        334 non-null    float64       
 4   Low         334 non-null    float64       
 5   Prev Close  334 non-null    float64       
 6   LTP         334 non-null    float64       
 7   Close       334 non-null    float64       
 8   VWAP        334 non-null    float64       
 9   52W H       334 non-null    float64       
 10  52W L       334 non-null    float64       
 11  Volume      334 non-null    int64         
 12  Value       334 non-null    float64       
 13  #Trades     334 non-null    int64         
dtypes: datetime64[ns](1), float64(10), int64(2), object(1)
memory usage: 53.3 KB


## Feature engineering

### Daily candle based columns

In [7]:
stock_df['Range'] = stock_df['High'] - stock_df['Low']
stock_df['IsGreen'] = (stock_df['Close'] >= stock_df['Prev Close']).astype(int)
stock_df['Is52WLow'] = np.isclose(stock_df['Low'], stock_df['52W L'], atol = 0, rtol = 1e-4).astype(int)
stock_df['Is52WHigh'] = np.isclose(stock_df['High'], stock_df['52W H'], atol = 0, rtol = 1e-4).astype(int)
stock_df = stock_df.drop(columns = ['Series', 'Prev Close'])
stock_df.sample(10)

,Date,Open,High,Low,LTP,Close,VWAP,52W H,52W L,Volume,Value,#Trades,Range,IsGreen,Is52WLow,Is52WHigh
140,2023-09-15,124.95,124.95,123.03,123.50,123.40,123.54,127.35,85.00,72840,8.998978e+06,1345,1.92,1,0,0
151,2023-10-04,123.43,123.99,121.52,123.00,122.22,122.48,127.52,85.00,97069,1.188932e+07,2422,2.47,0,0,0
215,2024-01-05,143.93,144.69,143.20,144.43,144.21,144.12,144.69,85.00,238677,3.439697e+07,2527,1.49,1,0,1
173,2023-11-06,126.70,126.75,123.17,124.60,124.39,124.18,128.70,85.00,176871,2.196337e+07,1240,3.58,1,0,0
182,2023-11-17,129.96,130.79,129.57,130.50,130.10,130.43,130.79,85.00,216775,2.827472e+07,1615,1.22,1,0,1
166,2023-10-26,119.91,120.00,115.00,119.95,118.82,116.99,128.70,85.00,301071,3.522354e+07,3798,5.00,0,0,0
259,2024-03-07,149.29,149.29,143.99,148.98,148.88,147.89,158.62,85.00,164074,2.426546e+07,2914,5.30,1,0,0
327,2024-06-20,171.05,171.77,169.16,171.25,171.35,171.04,176.00,103.00,659333,1.127733e+08,4148,2.61,1,0,0
23,2023-03-27,88.99,88.99,86.45,86.75,86.53,86.90,99.20,86.45,19905,1.729735e+06,386,2.54,0,1,0
178,2023-11-12,128.95,128.95,126.69,127.50,127.61,127.52,128.95,85.00,81375,1.037707e+07,1570,2.26,1,0,1


In [8]:
((stock_df[['IsGreen', 'Is52WLow', 'Is52WHigh']].sum() * 100) / len(stock_df)).round(2)

IsGreen      67.37
Is52WLow      2.10
Is52WHigh    18.86
dtype: float64

In [9]:
stock_df.shape

(334, 16)

### Streak

In [10]:
stock_df["Streak"] = (stock_df["IsGreen"] != stock_df["IsGreen"].shift(1)).cumsum()
stock_df["Streak"] = stock_df.groupby("Streak").cumcount() + 1
stock_df

,Date,Open,High,Low,LTP,Close,VWAP,52W H,52W L,Volume,Value,#Trades,Range,IsGreen,Is52WLow,Is52WHigh,Streak
0,2023-02-21,92.00,94.00,92.00,93.50,93.48,93.22,94.0,92.00,2565,239114.97,38,2.00,1,1,1,1
1,2023-02-22,93.48,93.50,90.74,91.75,92.02,92.25,94.0,90.74,12207,1126142.91,172,2.76,0,1,0,1
2,2023-02-23,92.02,92.02,89.75,90.50,90.59,90.76,94.0,89.75,6153,558445.34,102,2.27,0,1,0,2
3,2023-02-24,90.76,90.76,90.50,90.60,90.59,90.62,94.0,89.75,1944,176173.22,47,0.26,1,0,0,1
4,2023-02-27,92.50,95.80,87.50,89.85,89.97,89.74,95.8,87.50,6801,610333.75,227,8.30,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329,2024-06-24,172.49,172.49,169.05,171.21,170.92,170.91,176.0,103.40,300649,51383833.52,6275,3.44,0,0,0,2
330,2024-06-25,171.78,173.06,170.92,171.30,171.42,171.94,176.0,103.40,381223,65548651.83,4284,2.14,1,0,0,1
331,2024-06-26,171.43,172.37,171.00,171.35,171.45,171.54,176.0,103.40,230914,39610803.32,4389,1.37,1,0,0,2
332,2024-06-27,171.54,172.54,169.45,170.70,170.42,170.80,176.0,103.40,447571,76442930.84,7430,3.09,0,0,0,1


### Date based columns

In [11]:
stock_df['Day'] = stock_df['Date'].dt.day
stock_df['Month'] = stock_df['Date'].dt.month
stock_df['Year'] = stock_df['Date'].dt.year
stock_df['Weekday'] = stock_df['Date'].dt.weekday
stock_df['DayOfYear'] = stock_df['Date'].dt.day_of_year
stock_df['Quarter'] = stock_df['Date'].dt.quarter
stock_df['DaysSinceLastTradingSession'] = (stock_df['Date'] - stock_df['Date'].shift(1)).dt.days.fillna(1).astype(int)

stock_df.sample(10)

,Date,Open,High,Low,LTP,Close,VWAP,52W H,52W L,Volume,Value,#Trades,Range,IsGreen,Is52WLow,Is52WHigh,Streak,Day,Month,Year,Weekday,DayOfYear,Quarter,DaysSinceLastTradingSession
153,2023-10-06,123.49,124.00,122.61,123.60,123.55,123.50,127.52,85.0,80245,9910650.52,933,1.39,1,0,0,2,6,10,2023,4,279,4,1
10,2023-03-08,92.90,99.20,92.01,92.50,92.49,93.57,99.20,87.5,29541,2764130.77,382,7.19,1,0,1,6,8,3,2023,2,67,1,2
49,2023-05-09,95.95,96.52,95.60,95.67,95.69,96.08,100.00,85.0,55207,5304241.08,285,0.92,0,0,0,1,9,5,2023,1,129,2,1
38,2023-04-21,91.62,91.81,91.40,91.65,91.62,91.61,99.20,85.0,9587,878274.93,87,0.41,0,0,0,1,21,4,2023,4,111,2,1
208,2023-12-27,139.06,139.59,138.41,139.31,139.31,139.30,143.25,85.0,117241,16331781.73,1860,1.18,1,0,0,1,27,12,2023,2,361,4,1
328,2024-06-21,174.40,174.40,170.91,171.56,171.28,171.61,176.00,103.0,358620,61541066.54,5264,3.49,0,0,0,1,21,6,2024,4,173,2,1
171,2023-11-02,124.05,124.05,120.72,121.99,121.62,121.34,128.70,85.0,75595,9173053.39,1026,3.33,1,0,0,1,2,11,2023,3,306,4,1
202,2023-12-18,139.50,140.10,138.22,139.50,139.61,139.46,140.10,85.0,173919,24254568.42,2092,1.88,1,0,1,4,18,12,2023,0,352,4,3
126,2023-08-28,118.90,118.90,115.91,116.90,116.78,116.66,119.08,85.0,49017,5718484.34,874,2.99,1,0,0,1,28,8,2023,0,240,3,3
39,2023-04-24,93.45,93.45,91.76,92.18,92.06,92.02,99.20,85.0,3842,353543.42,114,1.69,1,0,0,1,24,4,2023,0,114,2,3


In [12]:
stock_df.shape

(334, 24)

### Moving averages

#### `Close`

In [13]:
for window in cnst.ROLLING_WINDOWS:
    stock_df[f'Close {window}MA'] = stock_df['Close'].rolling(window = window, min_periods = 1).mean().round(2)

stock_df.filter(regex = "Close.*")

,Close,Close 7MA,Close 15MA,Close 30MA,Close 60MA
0,93.48,93.48,93.48,93.48,93.48
1,92.02,92.75,92.75,92.75,92.75
2,90.59,92.03,92.03,92.03,92.03
3,90.59,91.67,91.67,91.67,91.67
4,89.97,91.33,91.33,91.33,91.33
...,...,...,...,...,...
329,170.92,170.57,165.26,161.29,157.20
330,171.42,170.76,165.99,161.96,157.68
331,171.45,171.06,166.94,162.56,158.14
332,170.42,170.99,168.08,163.07,158.58


#### `Range`

In [14]:
for window in cnst.ROLLING_WINDOWS:
    stock_df[f'Range {window}MA'] = stock_df['Range'].rolling(window = window, min_periods = 1).mean().round(2)

stock_df.filter(regex = "Range.*")

,Range,Range 7MA,Range 15MA,Range 30MA,Range 60MA
0,2.00,2.00,2.00,2.00,2.00
1,2.76,2.38,2.38,2.38,2.38
2,2.27,2.34,2.34,2.34,2.34
3,0.26,1.82,1.82,1.82,1.82
4,8.30,3.12,3.12,3.12,3.12
...,...,...,...,...,...
329,3.44,4.60,5.20,4.54,4.08
330,2.14,4.21,5.07,4.43,4.07
331,1.37,3.38,4.30,4.33,4.07
332,3.09,3.39,3.72,4.28,4.10


#### `VWAP`

In [15]:
for window in cnst.ROLLING_WINDOWS:
    stock_df[f'VWAP {window}MA'] = stock_df['VWAP'].rolling(window = window, min_periods = 1).mean().round(2)

stock_df.filter(regex = "VWAP.*")

,VWAP,VWAP 7MA,VWAP 15MA,VWAP 30MA,VWAP 60MA
0,93.22,93.22,93.22,93.22,93.22
1,92.25,92.74,92.74,92.74,92.74
2,90.76,92.08,92.08,92.08,92.08
3,90.62,91.71,91.71,91.71,91.71
4,89.74,91.32,91.32,91.32,91.32
...,...,...,...,...,...
329,170.91,170.37,164.76,161.00,156.99
330,171.94,170.79,165.53,161.73,157.48
331,171.54,171.09,166.66,162.34,157.95
332,170.80,171.17,167.96,162.86,158.39


#### `Volume`

In [16]:
for window in cnst.ROLLING_WINDOWS:
    stock_df[f'Volume {window}MA'] = stock_df['Volume'].rolling(window = window, min_periods = 1).mean().astype(int)

stock_df.filter(regex = "Volume.*")

,Volume,Volume 7MA,Volume 15MA,Volume 30MA,Volume 60MA
0,2565,2565,2565,2565,2565
1,12207,7386,7386,7386,7386
2,6153,6975,6975,6975,6975
3,1944,5717,5717,5717,5717
4,6801,5934,5934,5934,5934
...,...,...,...,...,...
329,300649,512428,813389,591624,496486
330,381223,496526,787784,587682,497101
331,230914,421693,681559,587897,495512
332,447571,424631,631392,577190,497577


#### `Value`

In [17]:
for window in cnst.ROLLING_WINDOWS:
    stock_df[f'Value {window}MA'] = stock_df['Value'].rolling(window = window, min_periods = 1).mean().astype(int)

stock_df.filter(regex = "Value.*")

,Value,Value 7MA,Value 15MA,Value 30MA,Value 60MA
0,239114.97,239114,239114,239114,239114
1,1126142.91,682628,682628,682628,682628
2,558445.34,641234,641234,641234,641234
3,176173.22,524969,524969,524969,524969
4,610333.75,542042,542042,542042,542042
...,...,...,...,...,...
329,51383833.52,87246993,131889156,94982868,78169393
330,65548651.83,84718957,128076678,94668479,78444683
331,39610803.32,72109486,111910940,94843635,78324383
332,76442930.84,72642843,104895072,93415120,78819404


#### `#Trades`

In [18]:
for window in cnst.ROLLING_WINDOWS:
    stock_df[f'#Trades {window}MA'] = stock_df['#Trades'].rolling(window = window, min_periods = 1).mean().astype(int)

stock_df.filter(regex = "#Trades.*")

,#Trades,#Trades 7MA,#Trades 15MA,#Trades 30MA,#Trades 60MA
0,38,38,38,38,38
1,172,105,105,105,105
2,102,104,104,104,104
3,47,89,89,89,89
4,227,117,117,117,117
...,...,...,...,...,...
329,6275,6680,8299,6552,5943
330,4284,6222,8103,6355,5929
331,4389,5534,6786,6395,5947
332,7430,5687,6116,6520,6013


### Target columns

In [19]:
for window in cnst.TARGET_WINDOWS:
    stock_df[f'Target {window}D'] = stock_df['Close'].shift(-window)

stock_df.filter(regex = "Target.*")

,Target 3D,Target 7D,Target 15D,Target 30D
0,90.59,91.00,89.33,89.94
1,89.97,91.48,88.68,90.61
2,90.75,92.45,88.88,91.09
3,90.85,92.49,89.17,91.28
4,91.00,92.43,88.70,91.62
...,...,...,...,...
329,170.42,NaN,NaN,NaN
330,172.12,NaN,NaN,NaN
331,NaN,NaN,NaN,NaN
332,NaN,NaN,NaN,NaN


### Saving processed data

In [20]:
stock_df.to_parquet(
    cnst.PROCESSED_DATA_DIR.joinpath(f'{STOCK_SYMBOL}-processed.parquet'), 
    index = False
)
stock_df

,Date,Open,High,Low,LTP,Close,VWAP,52W H,52W L,Volume,Value,#Trades,Range,IsGreen,Is52WLow,Is52WHigh,Streak,Day,Month,Year,Weekday,DayOfYear,Quarter,DaysSinceLastTradingSession,Close 7MA,Close 15MA,Close 30MA,Close 60MA,Range 7MA,Range 15MA,Range 30MA,Range 60MA,VWAP 7MA,VWAP 15MA,VWAP 30MA,VWAP 60MA,Volume 7MA,Volume 15MA,Volume 30MA,Volume 60MA,Value 7MA,Value 15MA,Value 30MA,Value 60MA,#Trades 7MA,#Trades 15MA,#Trades 30MA,#Trades 60MA,Target 3D,Target 7D,Target 15D,Target 30D
0,2023-02-21,92.00,94.00,92.00,93.50,93.48,93.22,94.0,92.00,2565,239114.97,38,2.00,1,1,1,1,21,2,2023,1,52,1,1,93.48,93.48,93.48,93.48,2.00,2.00,2.00,2.00,93.22,93.22,93.22,93.22,2565,2565,2565,2565,239114,239114,239114,239114,38,38,38,38,90.59,91.00,89.33,89.94
1,2023-02-22,93.48,93.50,90.74,91.75,92.02,92.25,94.0,90.74,12207,1126142.91,172,2.76,0,1,0,1,22,2,2023,2,53,1,1,92.75,92.75,92.75,92.75,2.38,2.38,2.38,2.38,92.74,92.74,92.74,92.74,7386,7386,7386,7386,682628,682628,682628,682628,105,105,105,105,89.97,91.48,88.68,90.61
2,2023-02-23,92.02,92.02,89.75,90.50,90.59,90.76,94.0,89.75,6153,558445.34,102,2.27,0,1,0,2,23,2,2023,3,54,1,1,92.03,92.03,92.03,92.03,2.34,2.34,2.34,2.34,92.08,92.08,92.08,92.08,6975,6975,6975,6975,641234,641234,641234,641234,104,104,104,104,90.75,92.45,88.88,91.09
3,2023-02-24,90.76,90.76,90.50,90.60,90.59,90.62,94.0,89.75,1944,176173.22,47,0.26,1,0,0,1,24,2,2023,4,55,1,1,91.67,91.67,91.67,91.67,1.82,1.82,1.82,1.82,91.71,91.71,91.71,91.71,5717,5717,5717,5717,524969,524969,524969,524969,89,89,89,89,90.85,92.49,89.17,91.28
4,2023-02-27,92.50,95.80,87.50,89.85,89.97,89.74,95.8,87.50,6801,610333.75,227,8.30,0,1,1,1,27,2,2023,0,58,1,3,91.33,91.33,91.33,91.33,3.12,3.12,3.12,3.12,91.32,91.32,91.32,91.32,5934,5934,5934,5934,542042,542042,542042,542042,117,117,117,117,91.00,92.43,88.70,91.62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329,2024-06-24,172.49,172.49,169.05,171.21,170.92,170.91,176.0,103.40,300649,51383833.52,6275,3.44,0,0,0,2,24,6,2024,0,176,2,3,170.57,165.26,161.29,157.20,4.60,5.20,4.54,4.08,170.37,164.76,161.00,156.99,512428,813389,591624,496486,87246993,131889156,94982868,78169393,6680,8299,6552,5943,170.42,NaN,NaN,NaN
330,2024-06-25,171.78,173.06,170.92,171.30,171.42,171.94,176.0,103.40,381223,65548651.83,4284,2.14,1,0,0,1,25,6,2024,1,177,2,1,170.76,165.99,161.96,157.68,4.21,5.07,4.43,4.07,170.79,165.53,161.73,157.48,496526,787784,587682,497101,84718957,128076678,94668479,78444683,6222,8103,6355,5929,172.12,NaN,NaN,NaN
331,2024-06-26,171.43,172.37,171.00,171.35,171.45,171.54,176.0,103.40,230914,39610803.32,4389,1.37,1,0,0,2,26,6,2024,2,178,2,1,171.06,166.94,162.56,158.14,3.38,4.30,4.33,4.07,171.09,166.66,162.34,157.95,421693,681559,587897,495512,72109486,111910940,94843635,78324383,5534,6786,6395,5947,NaN,NaN,NaN,NaN
332,2024-06-27,171.54,172.54,169.45,170.70,170.42,170.80,176.0,103.40,447571,76442930.84,7430,3.09,0,0,0,1,27,6,2024,3,179,2,1,170.99,168.08,163.07,158.58,3.39,3.72,4.28,4.10,171.17,167.96,162.86,158.39,424631,631392,577190,497577,72642843,104895072,93415120,78819404,5687,6116,6520,6013,NaN,NaN,NaN,NaN


## Data standardization

### Stock price based columns

In [21]:
stock_price_cols = [
    'Open', 'High', 'Low', 'LTP', '52W H', '52W L'
] + (
    stock_df.filter(regex = "Close.*").columns.to_list() +
    stock_df.filter(regex = "Range.*").columns.to_list() +
    stock_df.filter(regex = "VWAP.*").columns.to_list() +
    stock_df.filter(regex = "Target.*").columns.to_list()
)

stock_price_cols

['Open',
 'High',
 'Low',
 'LTP',
 '52W H',
 '52W L',
 'Close',
 'Close 7MA',
 'Close 15MA',
 'Close 30MA',
 'Close 60MA',
 'Range',
 'Range 7MA',
 'Range 15MA',
 'Range 30MA',
 'Range 60MA',
 'VWAP',
 'VWAP 7MA',
 'VWAP 15MA',
 'VWAP 30MA',
 'VWAP 60MA',
 'Target 3D',
 'Target 7D',
 'Target 15D',
 'Target 30D']

In [22]:
stock_df[stock_price_cols] = stock_df[stock_price_cols].div(stock_df['Close'], axis = 0).round(3)
stock_df[stock_price_cols]

,Open,High,Low,LTP,52W H,52W L,Close,Close 7MA,Close 15MA,Close 30MA,Close 60MA,Range,Range 7MA,Range 15MA,Range 30MA,Range 60MA,VWAP,VWAP 7MA,VWAP 15MA,VWAP 30MA,VWAP 60MA,Target 3D,Target 7D,Target 15D,Target 30D
0,0.984,1.006,0.984,1.000,1.006,0.984,1.0,1.000,1.000,1.000,1.000,0.021,0.021,0.021,0.021,0.021,0.997,0.997,0.997,0.997,0.997,0.969,0.973,0.956,0.962
1,1.016,1.016,0.986,0.997,1.022,0.986,1.0,1.008,1.008,1.008,1.008,0.030,0.026,0.026,0.026,0.026,1.002,1.008,1.008,1.008,1.008,0.978,0.994,0.964,0.985
2,1.016,1.016,0.991,0.999,1.038,0.991,1.0,1.016,1.016,1.016,1.016,0.025,0.026,0.026,0.026,0.026,1.002,1.016,1.016,1.016,1.016,1.002,1.021,0.981,1.006
3,1.002,1.002,0.999,1.000,1.038,0.991,1.0,1.012,1.012,1.012,1.012,0.003,0.020,0.020,0.020,0.020,1.000,1.012,1.012,1.012,1.012,1.003,1.021,0.984,1.008
4,1.028,1.065,0.973,0.999,1.065,0.973,1.0,1.015,1.015,1.015,1.015,0.092,0.035,0.035,0.035,0.035,0.997,1.015,1.015,1.015,1.015,1.011,1.027,0.986,1.018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329,1.009,1.009,0.989,1.002,1.030,0.605,1.0,0.998,0.967,0.944,0.920,0.020,0.027,0.030,0.027,0.024,1.000,0.997,0.964,0.942,0.918,0.997,NaN,NaN,NaN
330,1.002,1.010,0.997,0.999,1.027,0.603,1.0,0.996,0.968,0.945,0.920,0.012,0.025,0.030,0.026,0.024,1.003,0.996,0.966,0.943,0.919,1.004,NaN,NaN,NaN
331,1.000,1.005,0.997,0.999,1.027,0.603,1.0,0.998,0.974,0.948,0.922,0.008,0.020,0.025,0.025,0.024,1.001,0.998,0.972,0.947,0.921,NaN,NaN,NaN,NaN
332,1.007,1.012,0.994,1.002,1.033,0.607,1.0,1.003,0.986,0.957,0.931,0.018,0.020,0.022,0.025,0.024,1.002,1.004,0.986,0.956,0.929,NaN,NaN,NaN,NaN


### Volume based columns

In [23]:
volume_cols = stock_df.filter(regex = "Volume.*").columns.to_list()

volume_cols

['Volume', 'Volume 7MA', 'Volume 15MA', 'Volume 30MA', 'Volume 60MA']

In [24]:
stock_df[volume_cols] = stock_df[volume_cols].div(stock_df['Volume'], axis = 0).round(3)
stock_df[volume_cols]

,Volume,Volume 7MA,Volume 15MA,Volume 30MA,Volume 60MA
0,1.0,1.000,1.000,1.000,1.000
1,1.0,0.605,0.605,0.605,0.605
2,1.0,1.134,1.134,1.134,1.134
3,1.0,2.941,2.941,2.941,2.941
4,1.0,0.873,0.873,0.873,0.873
...,...,...,...,...,...
329,1.0,1.704,2.705,1.968,1.651
330,1.0,1.302,2.066,1.542,1.304
331,1.0,1.826,2.952,2.546,2.146
332,1.0,0.949,1.411,1.290,1.112


### Value based columns

In [25]:
value_cols = stock_df.filter(regex = "Value.*").columns.to_list()

value_cols

['Value', 'Value 7MA', 'Value 15MA', 'Value 30MA', 'Value 60MA']

In [26]:
stock_df[value_cols] = stock_df[value_cols].div(stock_df['Value'], axis = 0).round(3)
stock_df[value_cols]

,Value,Value 7MA,Value 15MA,Value 30MA,Value 60MA
0,1.0,1.000,1.000,1.000,1.000
1,1.0,0.606,0.606,0.606,0.606
2,1.0,1.148,1.148,1.148,1.148
3,1.0,2.980,2.980,2.980,2.980
4,1.0,0.888,0.888,0.888,0.888
...,...,...,...,...,...
329,1.0,1.698,2.567,1.848,1.521
330,1.0,1.292,1.954,1.444,1.197
331,1.0,1.820,2.825,2.394,1.977
332,1.0,0.950,1.372,1.222,1.031


### Trade count based columns

In [27]:
trade_count_cols = stock_df.filter(regex = "#Trades.*").columns.to_list()

trade_count_cols

['#Trades', '#Trades 7MA', '#Trades 15MA', '#Trades 30MA', '#Trades 60MA']

In [28]:
stock_df[trade_count_cols] = stock_df[trade_count_cols].div(stock_df['#Trades'], axis = 0).round(3)
stock_df[trade_count_cols]

,#Trades,#Trades 7MA,#Trades 15MA,#Trades 30MA,#Trades 60MA
0,1.0,1.000,1.000,1.000,1.000
1,1.0,0.610,0.610,0.610,0.610
2,1.0,1.020,1.020,1.020,1.020
3,1.0,1.894,1.894,1.894,1.894
4,1.0,0.515,0.515,0.515,0.515
...,...,...,...,...,...
329,1.0,1.065,1.323,1.044,0.947
330,1.0,1.452,1.891,1.483,1.384
331,1.0,1.261,1.546,1.457,1.355
332,1.0,0.765,0.823,0.878,0.809


### Dropping unnecessary columns

In [29]:
stock_df = stock_df.drop(columns = ['Date', 'Close', 'Range', 'Volume', 'Value', '#Trades'])
stock_df

,Open,High,Low,LTP,VWAP,52W H,52W L,IsGreen,Is52WLow,Is52WHigh,Streak,Day,Month,Year,Weekday,DayOfYear,Quarter,DaysSinceLastTradingSession,Close 7MA,Close 15MA,Close 30MA,Close 60MA,Range 7MA,Range 15MA,Range 30MA,Range 60MA,VWAP 7MA,VWAP 15MA,VWAP 30MA,VWAP 60MA,Volume 7MA,Volume 15MA,Volume 30MA,Volume 60MA,Value 7MA,Value 15MA,Value 30MA,Value 60MA,#Trades 7MA,#Trades 15MA,#Trades 30MA,#Trades 60MA,Target 3D,Target 7D,Target 15D,Target 30D
0,0.984,1.006,0.984,1.000,0.997,1.006,0.984,1,1,1,1,21,2,2023,1,52,1,1,1.000,1.000,1.000,1.000,0.021,0.021,0.021,0.021,0.997,0.997,0.997,0.997,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,0.969,0.973,0.956,0.962
1,1.016,1.016,0.986,0.997,1.002,1.022,0.986,0,1,0,1,22,2,2023,2,53,1,1,1.008,1.008,1.008,1.008,0.026,0.026,0.026,0.026,1.008,1.008,1.008,1.008,0.605,0.605,0.605,0.605,0.606,0.606,0.606,0.606,0.610,0.610,0.610,0.610,0.978,0.994,0.964,0.985
2,1.016,1.016,0.991,0.999,1.002,1.038,0.991,0,1,0,2,23,2,2023,3,54,1,1,1.016,1.016,1.016,1.016,0.026,0.026,0.026,0.026,1.016,1.016,1.016,1.016,1.134,1.134,1.134,1.134,1.148,1.148,1.148,1.148,1.020,1.020,1.020,1.020,1.002,1.021,0.981,1.006
3,1.002,1.002,0.999,1.000,1.000,1.038,0.991,1,0,0,1,24,2,2023,4,55,1,1,1.012,1.012,1.012,1.012,0.020,0.020,0.020,0.020,1.012,1.012,1.012,1.012,2.941,2.941,2.941,2.941,2.980,2.980,2.980,2.980,1.894,1.894,1.894,1.894,1.003,1.021,0.984,1.008
4,1.028,1.065,0.973,0.999,0.997,1.065,0.973,0,1,1,1,27,2,2023,0,58,1,3,1.015,1.015,1.015,1.015,0.035,0.035,0.035,0.035,1.015,1.015,1.015,1.015,0.873,0.873,0.873,0.873,0.888,0.888,0.888,0.888,0.515,0.515,0.515,0.515,1.011,1.027,0.986,1.018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329,1.009,1.009,0.989,1.002,1.000,1.030,0.605,0,0,0,2,24,6,2024,0,176,2,3,0.998,0.967,0.944,0.920,0.027,0.030,0.027,0.024,0.997,0.964,0.942,0.918,1.704,2.705,1.968,1.651,1.698,2.567,1.848,1.521,1.065,1.323,1.044,0.947,0.997,NaN,NaN,NaN
330,1.002,1.010,0.997,0.999,1.003,1.027,0.603,1,0,0,1,25,6,2024,1,177,2,1,0.996,0.968,0.945,0.920,0.025,0.030,0.026,0.024,0.996,0.966,0.943,0.919,1.302,2.066,1.542,1.304,1.292,1.954,1.444,1.197,1.452,1.891,1.483,1.384,1.004,NaN,NaN,NaN
331,1.000,1.005,0.997,0.999,1.001,1.027,0.603,1,0,0,2,26,6,2024,2,178,2,1,0.998,0.974,0.948,0.922,0.020,0.025,0.025,0.024,0.998,0.972,0.947,0.921,1.826,2.952,2.546,2.146,1.820,2.825,2.394,1.977,1.261,1.546,1.457,1.355,NaN,NaN,NaN,NaN
332,1.007,1.012,0.994,1.002,1.002,1.033,0.607,0,0,0,1,27,6,2024,3,179,2,1,1.003,0.986,0.957,0.931,0.020,0.022,0.025,0.024,1.004,0.986,0.956,0.929,0.949,1.411,1.290,1.112,0.950,1.372,1.222,1.031,0.765,0.823,0.878,0.809,NaN,NaN,NaN,NaN


### Saving standardized data

In [30]:
stock_df.to_parquet(
    cnst.PROCESSED_DATA_DIR.joinpath(f'{STOCK_SYMBOL}-standardized.parquet'), 
    index = False
)
stock_df

,Open,High,Low,LTP,VWAP,52W H,52W L,IsGreen,Is52WLow,Is52WHigh,Streak,Day,Month,Year,Weekday,DayOfYear,Quarter,DaysSinceLastTradingSession,Close 7MA,Close 15MA,Close 30MA,Close 60MA,Range 7MA,Range 15MA,Range 30MA,Range 60MA,VWAP 7MA,VWAP 15MA,VWAP 30MA,VWAP 60MA,Volume 7MA,Volume 15MA,Volume 30MA,Volume 60MA,Value 7MA,Value 15MA,Value 30MA,Value 60MA,#Trades 7MA,#Trades 15MA,#Trades 30MA,#Trades 60MA,Target 3D,Target 7D,Target 15D,Target 30D
0,0.984,1.006,0.984,1.000,0.997,1.006,0.984,1,1,1,1,21,2,2023,1,52,1,1,1.000,1.000,1.000,1.000,0.021,0.021,0.021,0.021,0.997,0.997,0.997,0.997,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,0.969,0.973,0.956,0.962
1,1.016,1.016,0.986,0.997,1.002,1.022,0.986,0,1,0,1,22,2,2023,2,53,1,1,1.008,1.008,1.008,1.008,0.026,0.026,0.026,0.026,1.008,1.008,1.008,1.008,0.605,0.605,0.605,0.605,0.606,0.606,0.606,0.606,0.610,0.610,0.610,0.610,0.978,0.994,0.964,0.985
2,1.016,1.016,0.991,0.999,1.002,1.038,0.991,0,1,0,2,23,2,2023,3,54,1,1,1.016,1.016,1.016,1.016,0.026,0.026,0.026,0.026,1.016,1.016,1.016,1.016,1.134,1.134,1.134,1.134,1.148,1.148,1.148,1.148,1.020,1.020,1.020,1.020,1.002,1.021,0.981,1.006
3,1.002,1.002,0.999,1.000,1.000,1.038,0.991,1,0,0,1,24,2,2023,4,55,1,1,1.012,1.012,1.012,1.012,0.020,0.020,0.020,0.020,1.012,1.012,1.012,1.012,2.941,2.941,2.941,2.941,2.980,2.980,2.980,2.980,1.894,1.894,1.894,1.894,1.003,1.021,0.984,1.008
4,1.028,1.065,0.973,0.999,0.997,1.065,0.973,0,1,1,1,27,2,2023,0,58,1,3,1.015,1.015,1.015,1.015,0.035,0.035,0.035,0.035,1.015,1.015,1.015,1.015,0.873,0.873,0.873,0.873,0.888,0.888,0.888,0.888,0.515,0.515,0.515,0.515,1.011,1.027,0.986,1.018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329,1.009,1.009,0.989,1.002,1.000,1.030,0.605,0,0,0,2,24,6,2024,0,176,2,3,0.998,0.967,0.944,0.920,0.027,0.030,0.027,0.024,0.997,0.964,0.942,0.918,1.704,2.705,1.968,1.651,1.698,2.567,1.848,1.521,1.065,1.323,1.044,0.947,0.997,NaN,NaN,NaN
330,1.002,1.010,0.997,0.999,1.003,1.027,0.603,1,0,0,1,25,6,2024,1,177,2,1,0.996,0.968,0.945,0.920,0.025,0.030,0.026,0.024,0.996,0.966,0.943,0.919,1.302,2.066,1.542,1.304,1.292,1.954,1.444,1.197,1.452,1.891,1.483,1.384,1.004,NaN,NaN,NaN
331,1.000,1.005,0.997,0.999,1.001,1.027,0.603,1,0,0,2,26,6,2024,2,178,2,1,0.998,0.974,0.948,0.922,0.020,0.025,0.025,0.024,0.998,0.972,0.947,0.921,1.826,2.952,2.546,2.146,1.820,2.825,2.394,1.977,1.261,1.546,1.457,1.355,NaN,NaN,NaN,NaN
332,1.007,1.012,0.994,1.002,1.002,1.033,0.607,0,0,0,1,27,6,2024,3,179,2,1,1.003,0.986,0.957,0.931,0.020,0.022,0.025,0.024,1.004,0.986,0.956,0.929,0.949,1.411,1.290,1.112,0.950,1.372,1.222,1.031,0.765,0.823,0.878,0.809,NaN,NaN,NaN,NaN
